!!!! Для чекинга учеников - засабмитить препроцессинг с align, но заполнить пропуски импутером со средним - дает лучший результат скоров

n_estimators = 450, learning_rate= 0.065, colsample_bytree = 0.8, subsample = 0.8, max_depth = 4, reg_lambda = 0.998


Mean Absolute Error: 14642.478502247432

Root Mean Squared Error: 0.12153794263233741

Root Mean Squared Error: 29848.361709034547


In [1]:
import pandas as pd
import numpy as np
from math import *
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import OneHotEncoder
import warnings
warnings.filterwarnings('ignore')

In [41]:
# Read the data
X = pd.read_csv('/Users/avyny/Documents/Data_analyses-Kaggle/Data_House_Pricing/train.csv', index_col='Id')
X_test_full = pd.read_csv('/Users/avyny/Documents/Data_analyses-Kaggle/Data_House_Pricing/test.csv', index_col='Id')
# X = pd.read_csv('D:\\Learning_IT\\Data_analyses-Kaggle\\Data_House_Pricing\\train.csv', index_col='Id')
# X_test_full = pd.read_csv('D:\\Learning_IT\\Data_analyses-Kaggle\\Data_House_Pricing\\test.csv', index_col='Id')

y = X.SalePrice              
X.drop(['SalePrice'], axis=1, inplace=True)

X = X.drop(['Alley', 'PoolQC','Fence','MiscFeature', 'Utilities', 'Street', 'PoolArea', 'MiscVal'], axis=1)
X_test_full = X_test_full.drop(['Alley', 'PoolQC','Fence','MiscFeature', 'Utilities', 'Street', 'PoolArea', 'MiscVal'], axis=1)

# Break off validation set from training data
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=0)

# "Cardinality" means the number of unique values in a column
# Select categorical columns with relatively low cardinality (convenient but arbitrary)
# low_cardinality_cols = [cname for cname in X_train_full.columns if X_train_full[cname].nunique() < 10 and 
#                         X_train_full[cname].dtype == "object"]
# low_cardinality_cols = [cname for cname in X_train_full.columns if X_train_full[cname].dtype == "object"]

# # Select numeric columns
# numeric_cols = [cname for cname in X_train_full.columns if X_train_full[cname].dtype in ['int64', 'float64']]

# # Keep selected columns only
# my_cols = low_cardinality_cols + numeric_cols
# X_train = X_train_full[my_cols].copy()
# X_valid = X_valid_full[my_cols].copy()
# X_test = X_test_full[my_cols].copy()

In [42]:
### Get_dummies () without filling NA

# One-hot encode the data (to shorten the code, we use pandas)
# X_train = pd.get_dummies(X_train)
# X_valid = pd.get_dummies(X_valid)
# X_test = pd.get_dummies(X_test)
# X_train, X_valid = X_train.align(X_valid, join='left', axis=1)
# X_train, X_test = X_train.align(X_test, join='left', axis=1)

In [43]:
categorical_cols = [cname for cname in X.columns if X[cname].dtype == "object"]
# categorical_cols = [cname for cname in X.columns if X[cname].nunique() < 10 and 
#                         X[cname].dtype == "object"]
numerical_median = ['OverallCond', 'OverallQual', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2',
                   'BsmtUnfSF', 'LowQualFinSF', 'GrLivArea', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr',
                   'TotRmsAbvGrd', 'GarageYrBlt', 'GarageCars', 'EnclosedPorch', 'ScreenPorch', 'MoSold', 'YrSold'
                   ]

numerical_mean = ['LotFrontage', 'LotArea', '1stFlrSF', '2ndFlrSF', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF']

numerical_frequent = ['MSSubClass', 'BsmtFullBath', 'BsmtHalfBath', 'Fireplaces']

all_cols = numerical_median + numerical_mean + numerical_frequent + categorical_cols

X_train = X_train_full[all_cols].copy()
X_valid = X_valid_full[all_cols].copy()
X_test = X_test_full[all_cols].copy()

In [44]:
# numerical_transformer_mean = SimpleImputer(strategy='mean')
# numerical_transformer_median = SimpleImputer(strategy='median')
# numerical_transformer_freq = SimpleImputer(strategy='most_frequent')

# categorical_transformer = Pipeline(steps=[
#     ('imputer', SimpleImputer(strategy='most_frequent'))
# ])

# preprocessor = ColumnTransformer(
#     transformers=[
#         ('num1', numerical_transformer_mean, numerical_mean),
#         ('num2', numerical_transformer_median, numerical_median),
#         ('num3', numerical_transformer_freq, numerical_frequent),
#         ('cat', categorical_transformer, categorical_cols)
#     ])


In [45]:
# X_train1 = pd.DataFrame(preprocessor.fit_transform(X_train))
# X_train1.columns = numerical_mean + numerical_median + numerical_frequent + categorical_cols
# X_train1[numerical_median + numerical_mean + numerical_frequent] = X_train1[numerical_median + numerical_mean + numerical_frequent].apply(pd.to_numeric) 
# X_train1 = pd.get_dummies(X_train1)
# # X_train.head() # 263

# X_valid1 = pd.DataFrame(preprocessor.fit_transform(X_valid))
# X_valid1.columns = numerical_mean + numerical_median + numerical_frequent + categorical_cols
# X_valid1[numerical_median + numerical_mean + numerical_frequent] = X_valid1[numerical_median + numerical_mean + numerical_frequent].apply(pd.to_numeric) 
# X_valid1 = pd.get_dummies(X_valid1)
# # X_valid.head() # 239

# X_test1 = pd.DataFrame(preprocessor.fit_transform(X_test))
# X_test1.columns = numerical_mean + numerical_median + numerical_frequent + categorical_cols
# X_test1[numerical_median + numerical_mean + numerical_frequent] = X_test1[numerical_median + numerical_mean + numerical_frequent].apply(pd.to_numeric) 
# X_test1 = pd.get_dummies(X_test1)
# X_test1.index = X_test_full.index
# # X_test.head() # 252

# X_train1, X_valid1 = X_train1.align(X_valid1, join='left', axis=1)
# X_train1, X_test1 = X_train1.align(X_test1, join='left', axis=1)




### PART WITH SIMPLE IMMPUTER - ONE FOR ALL

X_train = pd.get_dummies(X_train)
X_valid = pd.get_dummies(X_valid)
X_test = pd.get_dummies(X_test)

si = SimpleImputer(strategy='mean')

X_train1 = pd.DataFrame(si.fit_transform(X_train))
X_train1.columns = X_train.columns

X_valid1 = pd.DataFrame(si.fit_transform(X_valid))
X_valid1.columns = X_valid.columns

X_test1 = pd.DataFrame(si.fit_transform(X_test))
X_test1.columns = X_test.columns
X_test1.index = X_test.index

X_train1, X_valid1 = X_train1.align(X_valid1, join='left', axis=1)
X_train1, X_test1 = X_train1.align(X_test1, join='left', axis=1)

In [46]:
my_model_2 = XGBRegressor(n_estimators = 450, learning_rate= 0.065, colsample_bytree = 0.8, subsample = 0.8, max_depth = 4, reg_lambda = 0.998)
# my_model_2 = XGBRegressor(n_estimators = 900, learning_rate= 0.04, subsample = 0.8)

# Fit the model
my_model_2.fit(X_train1, y_train)

[12:21:17] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.8, gamma=0,
             importance_type='gain', learning_rate=0.065, max_delta_step=0,
             max_depth=4, min_child_weight=1, missing=None, n_estimators=450,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=0.998, scale_pos_weight=1, seed=None,
             silent=None, subsample=0.8, verbosity=1)

***Detecting of feature importance***

In [61]:
pd.DataFrame({'x': range(0, 263), 'y': my_model_2.feature_importances_}).sort_values(by = 'y', ascending = False).head(7)

,x,y
16,16,0.186780
1,1,0.121367
169,169,0.116161
157,157,0.060454
10,10,0.045665
244,244,0.026188
9,9,0.025880


In [66]:
## The most important features for model are:

X_train1.iloc[:, [16, 1, 169, 157, 10, 244, 9]].head()

,GarageCars,OverallQual,BsmtQual_Ex,ExterQual_TA,FullBath,GarageCond_TA,GrLivArea
0,3.0,9.0,1.0,0.0,2.0,1.0,1828.0
1,1.0,5.0,0.0,1.0,1.0,1.0,894.0
2,2.0,5.0,0.0,1.0,1.0,1.0,964.0
3,3.0,8.0,0.0,0.0,2.0,1.0,1689.0
4,3.0,7.0,0.0,0.0,2.0,1.0,1541.0


In [47]:
# Get predictions
predictions_2 = my_model_2.predict(X_valid1)

mae_2 = mean_absolute_error(y_valid, predictions_2)
print("Mean Absolute Error:" , mae_2)

y_valid = np.array(y_valid)
# base = 2.5611
# # base = 2.494
# rmse = np.sqrt(mean_squared_error((np.log(predictions_2)/np.log(base)), (np.log(y_valid)/np.log(base))))
# rmse = np.sqrt(mean_squared_error(predictions_2, y_valid))
rmse = np.sqrt(mean_squared_error(np.log(predictions_2), np.log(y_valid)))
print("Root Mean Squared Error:" , rmse)

rmse2 = np.sqrt(mean_squared_error(predictions_2, y_valid))
print("Root Mean Squared Error:" , rmse2)

Mean Absolute Error: 15359.81144317209
Root Mean Squared Error: 0.1241015844976499
Root Mean Squared Error: 27368.303622670966


In [17]:
## SCORE2 = 0.13064 n_estimators = 450, learning_rate= 0.04, subsample = 0.5, max_depth = 5, colsample_bytree=0.8 + with droping vars
#    Mean Absolute Error: 15166.343161386987, Mean Squared Error: 27821.293682309726 

## + top2 SCORE = 14109.95424, SCORE2 = 0.12572  n_estimators = 450, learning_rate= 0.04, subsample = 0.5, max_depth = 5, colsample_bytree=0.8 + with dropping 
# except '3SsnPorch' (+)   Mean Absolute Error: 15142.2001953125; Mean Squared Error: 29109.281997806498

## - ?? the same model (previous) but with simple imputer align with median
# SCORE2 = 0.13415, Mean Absolute Error: 15094.940723994006, Root Mean Squared Error: 27935.43693200561

## - ?? the same model (previous) with pipeline and align
# SCORE2 = 0.13217 Mean Absolute Error: 15402.944162029109, Root Mean Squared Error: 28921.112689032045

## 0.1339 n_estimators = 450, learning_rate= 0.06, colsample_bytree = 0.81, subsample = 0.8, max_depth = 4 + with dropping 
# except '3SsnPorch' (+)   Mean Absolute Error: 15089.681145654966   Mean Squared Error: 30960.239737581527

## 0.1324   n_estimators = 450, learning_rate= 0.06, colsample_bytree = 0.8, subsample = 0.8, max_depth = 4 + with dropping 
# except '3SsnPorch' (+)   Mean Absolute Error: 14931.222977311643   Mean Squared Error: 29922.90346737125

## 0.1313  n_estimators = 450, learning_rate= 0.065, colsample_bytree = 0.8, subsample = 0.8, max_depth = 4 + with dropping 
# except '3SsnPorch' (+)   Mean Absolute Error: 14836.028119648972      Mean Squared Error: 30055.96683219649

## '+ top SCORE - 13997 SCORE2 - 0.13025 n_estimators = 450, learning_rate= 0.065, colsample_bytree = 0.8, subsample = 0.8, max_depth = 4, 
# reg_lambda = 0.998 + with dropping except '3SsnPorch' (+)
# Mean Absolute Error: 14757.579757063357     Mean Squared Error: 29908.293379567116

## '-' SCORE 14257 SCORE2 = 0.12945 0.1349 n_estimators = 450, learning_rate= 0.065, colsample_bytree = 0.8, subsample = 0.8, max_depth = 4, 
# reg_lambda = 0.998, min_child_weight = 3  + with dropping except '3SsnPorch' (+)
# Mean Absolute Error: 14884.822412778254    Mean Squared Error: 27713.851497170308

## '-' не улучшилось на 2м соревновании(можно попробовать, лучше чем предыущий максимальный скор)
## SCORE2 = 0.13040    n_estimators = 550, learning_rate= 0.065, colsample_bytree = 0.8, subsample = 0.8, max_depth = 4, 
# reg_lambda = 0.998 + with dropping except '3SsnPorch' (+)
# Mean Absolute Error: 14746.396002782534   Mean Squared Error: 29832.86481735542

In [180]:
## SCORE 14045 with dropping all wars and without limitation of objects
# n_estimators = 900, learning_rate= 0.04, subsample = 0.8 (MAE 15265, RMSE 30160)

## SCORE 14099 without dropping vars 
# n_estimators = 900, learning_rate= 0.04, subsample = 0.8 (MAE 15377, RMSE 30982)

## SCORE 14257 n_estimators = 450, learning_rate= 0.065, colsample_bytree = 0.8, subsample = 0.8, max_depth = 4, 
# reg_lambda = 0.998, min_child_weight = 3 (MAE 14884, RMSE 27713)

## SCORE 13997 n_estimators = 450, learning_rate= 0.065, colsample_bytree = 0.8, subsample = 0.8, max_depth = 4, 
# reg_lambda = 0.998 (MAE 14757, RMSE 29908)

In [41]:
## RESULTS

preds_test = my_model_2.predict(X_test1)
output = pd.DataFrame({'Id': X_test.index,
                       'SalePrice': preds_test})
output.to_csv('submissionNastya23.csv', index=False)